# RSA Encryption for Digital Signature

In [2]:
import random

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def mod_inverse(e, phi):
    d = 0
    x1, x2, y1 = 0, 1, 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi, e = e, temp2
        
        x = x2 - temp1 * x1
        y = d - temp1 * y1
        
        x2, x1 = x1, x
        d, y1 = y1, y
    
    if temp_phi == 1:
        return d + phi

def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError("Both numbers must be prime.")
    elif p == q:
        raise ValueError("p and q cannot be the same.")
    
    n = p * q
    phi = (p - 1) * (q - 1)

    e = random.randrange(1, phi)

    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    d = mod_inverse(e, phi)
    
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    key, n = pk
    cipher = [(ord(char) ** key) % n for char in plaintext]
    return cipher

def decrypt(pk, ciphertext):
    key, n = pk
    plain = [chr((char ** key) % n) for char in ciphertext]
    return ''.join(plain)

def sign(private_key, message):
    return encrypt(private_key, message)

def verify(public_key, message, signature):
    return message == decrypt(public_key, signature)

p = 61
q = 53

public, private = generate_keypair(p, q)
# print(public, private)

message = "World"
print("Original message:", message)

signature = sign(private, message)
print("Signature:", signature)

is_valid = verify(public, message, signature)
print("Signature valid:", is_valid)

Original message: World
Signature: [1429, 3039, 3042, 2426, 1076]
Signature valid: True


# Elliptic Curve Digital Signature

In [2]:
import random

p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
G = (Gx, Gy)
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def modinv(k, p):
    return pow(k, p - 2, p)

def point_add(P, Q, p):
    if P == (None, None):
        return Q
    if Q == (None, None):
        return P
    x1, y1 = P
    x2, y2 = Q
    if P == Q:
        m = (3 * x1 * x1 + a) * modinv(2 * y1, p) % p
    else:
        if x1 == x2 and y1 != y2:
            return (None, None)
        m = (y2 - y1) * modinv(x2 - x1, p) % p
    x3 = (m * m - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p
    return (x3, y3)

def scalar_mult(k, P, p):
    R = (None, None)
    addend = P
    while k:
        if k & 1:
            R = point_add(R, addend, p)
        addend = point_add(addend, addend, p)
        k >>= 1
    return R

def generate_keypair():
    private_key = random.randrange(1, n)
    public_key = scalar_mult(private_key, G, p)
    return private_key, public_key

def custom_hash(message):
    z = 0
    for char in message:
        z = (z << 8) + ord(char)
    return z % n

def sign(private_key, message):
    z = custom_hash(message)
    r = 0
    s = 0
    while r == 0 or s == 0:
        k = random.randrange(1, n)
        x, _ = scalar_mult(k, G, p)
        r = x % n
        s = ((z + r * private_key) * modinv(k, n)) % n
    return r, s

def verify(public_key, message, signature):
    r, s = signature
    if not (1 <= r < n and 1 <= s < n):
        return False
    z = custom_hash(message)
    w = modinv(s, n)
    u1 = (z * w) % n
    u2 = (r * w) % n
    X = point_add(scalar_mult(u1, G, p), scalar_mult(u2, public_key, p), p)
    if X == (None, None):
        return False
    x, _ = X
    return (x % n) == r

message = "Hello, AMAN"
private_key, public_key = generate_keypair()
signature = sign(private_key, message)
is_valid = verify(public_key, message, signature)
print(f"Private key: {private_key}")
print(f"Public key: {public_key}")
print(f"Signature: {signature}")
print(f"Verification: {is_valid}")

Private key: 7909884955600877510521008272855743970331510295459797785021972072985490677922
Public key: (55023621621222793492708005989069320826376463750368164808234206023846098139751, 52405776327216368156346414611443883344397661742997728757538427448965945628026)
Signature: (85748609281859947114094691417062521614861126648145558590706131797920087431469, 82019471739610934231525620027155406258602343691839039047150961812484390678101)
Verification: False


# Elliptic Curve Digital Signature

In [19]:
import random

P = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

A = 0
B = 7
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
G = (Gx, Gy)
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def mod_inverse(a, p):
    if a == 0:
        return 0
    lm, hm = 1, 0
    low, high = a % p, p
    while low > 1:
        ratio = high // low
        nm, new = hm - lm * ratio, high - low * ratio
        lm, low, hm, high = nm, new, lm, low
    return lm % p

def point_addition(P1, P2):
    if P1 == (None, None):
        return P2
    if P2 == (None, None):
        return P1

    x1, y1 = P1
    x2, y2 = P2

    if x1 == x2 and y1 != y2:
        return (None, None)

    if P1 != P2:
        m = (y2 - y1) * mod_inverse(x2 - x1, P) % P
    else:
        m = (3 * x1 * x1 + A) * mod_inverse(2 * y1, P) % P

    x3 = (m * m - x1 - x2) % P
    y3 = (m * (x1 - x3) - y1) % P
    return (x3, y3)

def point_multiplication(k, point):
    result = (None, None)
    addend = point

    while k:
        if k & 1:
            result = point_addition(result, addend)
        addend = point_addition(addend, addend)
        k >>= 1

    return result

def generate_keypair():
    private_key = random.randrange(1, N)
    public_key = point_multiplication(private_key, G)
    return private_key, public_key

def sign(message, private_key):
    z = int.from_bytes(message.encode(), 'big') % N
    r, s = 0, 0

    while r == 0 or s == 0:
        k = random.randrange(1, N)
        x, y = point_multiplication(k, G)
        r = x % N
        s = ((z + r * private_key) * mod_inverse(k, N)) % N

    return (r, s)

def verify(message, signature, public_key):
    z = int.from_bytes(message.encode(), 'big') % N
    r, s = signature

    if not (1 <= r <= N - 1) or not (1 <= s <= N - 1):
        return False

    w = mod_inverse(s, N)
    u1 = (z * w) % N
    u2 = (r * w) % N

    x, y = point_addition(point_multiplication(u1, G), point_multiplication(u2, public_key))

    return r == x % N

message = "Hello, KESHAV"
private_key, public_key = generate_keypair()

signature = sign(message, private_key)
print(f"Message: {message}")
print(f"Signature: {signature}")
print(f"Verification: {verify(message, signature, public_key)}")

Message: Hello, KESHAV
Signature: (44006925818301988798761787030623491404682445570144819132568423688567143736807, 44549040560010675704924297739186680315198318806542609108068811529941062821179)
Verification: True


In [2]:
def extended_gcd(a, b): 
    if b == 0:
        return a, 1, 0
    gcd, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return gcd, x, y

def mod_inverse(k, p):
    gcd, x, _ = extended_gcd(k, p)
    if gcd == 1:
        return x % p

def point_addition(P, Q, a, p): 
    if P == (0, 0):
        return Q
    if Q == (0, 0):
        return P
    if P == Q:
        m = (3 * P[0]**2 + a) * mod_inverse(2 * P[1], p) % p
    else:
        m = (Q[1] - P[1]) * mod_inverse(Q[0] - P[0], p) % p
    x = (m**2 - P[0] - Q[0]) % p
    y = (m * (P[0] - x) - P[1]) % p
    return (x, y)

def scalar_multiplication(k, P, a, p):
    R = (0, 0)
    while k > 0:
        if k % 2 == 1:
            R = point_addition(R, P, a, p)
        P = point_addition(P, P, a, p)
        k //= 2
    return R

p = 751
a = 1
b = 1
G = (0, 1)
n = 727
private_key = 123
public_key = scalar_multiplication(private_key, G, a, p)
message = 45
k = 456
R = scalar_multiplication(k, G, a, p)
r = R[0] % n
s = (mod_inverse(k, n) * (message + r * private_key)) % n
w = mod_inverse(s, n)
u1 = (message * w) % n
u2 = (r * w) % n
X = point_addition(scalar_multiplication(u1, G, a, p), scalar_multiplication(u2, public_key, a, p), a, p)
v = X[0] % n


print("Message:", message)
print("Signature: (r, s) =", (r, s))
print("Verification:" , v == r)

Message: 45
Signature: (r, s) = (477, 454)
Verification: True
